In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import imagenet_utils
import itertools
import shutil
import random
import glob
from keras.wrappers.scikit_learn import KerasClassifier
import sklearn
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from scipy import interp
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam,SGD
from keras.models import Sequential
from keras.models import Model, model_from_json
from keras.layers import Dense, Activation, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Flatten ,Dropout
from keras.regularizers import l2
from keras import regularizers, optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import glorot_normal, RandomNormal, Zeros
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc ,accuracy_score,precision_score,recall_score,f1_score,accuracy_score
import datetime, os
import time

In [7]:
def get_dataset():
    dataset = np.load('data.row.battah/ASD_process.npz') #Dataset ready in numpy array (removing background, resizing, and transforming into grayscale)
    X = dataset['X']
    y = dataset['y']

    mean = np.mean(X, axis=(0, 1, 2,3))
    std = np.std(X, axis=(0, 1, 2,3))
    X = (X - mean) / (std+1e-7)

    return X,y

In [9]:
def GetModel():
    
    model = Sequential()
    # Conv Block 1
    model.add(Conv2D(64, (3, 3), padding='same' , strides=(1, 1),kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal(),
                     input_shape=X[train].shape[1:]))
    model.add(BatchNormalization(momentum=0.05,trainable=True, epsilon=1e-05))# affine=True, track_running_stats=True
    model.add(Activation(activation='relu'))#(inplace)

    model.add(Conv2D(128, (3, 3), padding='same',strides=(1, 1),kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization(momentum=0.05, epsilon=1e-05))
    model.add(Activation(activation='relu'))#(inplace)

    model.add(Conv2D(128, (3, 3), padding='same',strides=(1, 1),kernel_regularizer=regularizers.l2(0.005), kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization(momentum=0.05, epsilon=1e-05))
    model.add(Activation(activation='relu'))#(inplace)
     
    model.add(Conv2D(128, (3, 3), padding='same',strides=(1, 1),kernel_regularizer=regularizers.l2(0.005), kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization(momentum=0.05, epsilon=1e-05))
    model.add(Activation(activation='relu'))#(inplace)
     
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))# padding=0, dilation=(1, 1), ceil_mode=False)
    model.add(Dropout(0.1))          


    # Conv Block 2
    model.add(Conv2D(128, (3, 3), padding='same',strides=(1, 1),kernel_regularizer=regularizers.l2(0.005), kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization(momentum=0.05, epsilon=1e-05))
    model.add(Activation(activation='relu'))#(inplace)

    model.add(Conv2D(128, (3, 3), padding='same',strides=(1, 1),kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization(momentum=0.05, epsilon=1e-05))
    model.add(Activation(activation='relu'))#(inplace)

    model.add(Conv2D(256, (3, 3), padding='same',strides=(1, 1),kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization(momentum=0.05, epsilon=1e-05))
    model.add(Activation(activation='relu'))#(inplace)
     
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))# padding=0, dilation=(1, 1), ceil_mode=False)
    model.add(Dropout(0.1))


    # Conv Block 3
    model.add(Conv2D(256, (3, 3), padding='same',strides=(1, 1),kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization(momentum=0.05, epsilon=1e-05))
    model.add(Activation(activation='relu'))#(inplace)

    model.add(Conv2D(256, (3, 3), padding='same',strides=(1, 1),kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization(momentum=0.05, epsilon=1e-05))
    model.add(Activation(activation='relu'))#(inplace)
     
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))# padding=0, dilation=(1, 1), ceil_mode=False)
    model.add(Dropout(0.1))

    # Conv Block 4
    model.add(Conv2D(512, (3, 3), padding='same',strides=(1, 1),kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization(momentum=0.05, epsilon=1e-05))
    model.add(Activation(activation='relu'))#(inplace)
     
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))# padding=0, dilation=(1, 1), ceil_mode=False)
    model.add(Dropout(0.1))

    # Conv Block 5
    model.add(Conv2D(2048, (1, 1), padding='same',strides=(1, 1),kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization(momentum=0.05, epsilon=1e-05))
    model.add(Activation(activation='relu'))#(inplace)

    model.add(Conv2D(256, (1, 1), padding='same',strides=(1, 1),kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization(momentum=0.05, epsilon=1e-05))
    model.add(Activation(activation='relu'))#(inplace)
     
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))# padding=0, dilation=(1, 1), ceil_mode=False)
    model.add(Dropout(0.1))


    # Conv Block 6
    model.add(Conv2D(256, (3, 3), padding='same',strides=(1, 1),kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization(momentum=0.05, epsilon=1e-05))
    model.add(Activation(activation='relu'))#(inplace)


    # Classifier
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    BATCH_SIZE =64
    STEPS_PER_EPOCH =len(X[train])//BATCH_SIZE 
    step = tf.Variable(0, trainable=False)
    boundaries = [15*STEPS_PER_EPOCH,30*STEPS_PER_EPOCH,
              40*STEPS_PER_EPOCH]
    values = [0.1, 0.01 , 0.001, 0.0001]
    learning_rate_fn = keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries, values)


    opt = SGD(learning_rate=learning_rate_fn(step))

    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [11]:
def train_cross_validate(folds = 3):
    histories = []
    models = []

    X, y = get_dataset()

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 3)
    kfold = StratifiedKFold(n_splits=nfolds, shuffle=True, random_state=1).split(X, y) 
    


    for train, test in kfold:
        model = GetModel()
        history = model.fit(X[train], y[train], validation_split=0.3,batch_size=64,
                  epochs=50,steps_per_epoch = len(X[train])//64,validation_steps = len([test])//64,
                   validation_data=(X[test], y[test]),shuffle=True)
       
        models.append(model)
        histories.append(history)
        
    return histories, models

In [14]:
def predict(folds = 3):
   X, y = get_dataset()
   histories, models = train_cross_validate(folds = folds)
   

   scores=[]
   auc =[]
   tprs = []
   aucs = []
   accuracy =[]
   recall =[]
   precision = []
   mean_fpr = np.linspace(0, 1, 100)

    

   for i in range(nfolds):
     result = models[i].evaluate(X[test], y[test], verbose=0)
     scores.append(result)
     pred = models[i].predict(X[test])
     AUC = roc_auc_score(y[test], pred.round())
     auc.append(AUC)
     recall.append( recall_score(y[test], pred.round()) )
     precision.append( precision_score(y[test], pred.round()) )
     accuracy.append(accuracy_score(y[test], pred.round()))
   return precision, auc ,accuracy,recall

In [15]:
precision, auc ,accuracy,recall= predict(folds = 3)

NameError: ignored